Importing the modules needed for the project

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist
import haversine as hs
from scipy.spatial.distance import cdist
from python_tsp.heuristics import solve_tsp_local_search


Collecting the data (change the name of the file for another data)

In [ ]:

data_csv = pd.read_csv("clean_data.csv")
df = pd.DataFrame(data_csv)


Cleaning the data

In [ ]:
df.drop_duplicates(subset=['CourierId','Latitude','Longitude'],keep='first',inplace=True)
df = df.dropna(axis=0)
index_list = [i for i in range(len(df))]
df = df.set_index([index_list])
df

Looping the algorithm for every courier (without adding the distances to the dataframe)

In [ ]:
for i in set(df.CourierId):
    print("Current ID :",i)
    new_df = df.loc[df.CourierId == i]
    new_df = new_df.set_index([[i for i in range(len(new_df))]])
    dist_matrix = [[0]*len(new_df) for _ in range(len(new_df))]

    for j in new_df.index:
        for k in new_df.index:
            loc1=(new_df.Latitude[j], new_df.Longitude[j])
            loc2=(new_df.Latitude[k], new_df.Longitude[k])
            dist_matrix[j][k] = hs.haversine(loc1,loc2)
    
    dist_matrix = np.array(dist_matrix)

    arr1,rl_dis = solve_tsp_local_search(dist_matrix)
    traj =' -> '.join([str(b) for b in arr1])
    
    print("For Courier with ID:"+str(i)+",\nThe trajectory is :",traj)
    print("The distance is :",rl_dis,"km")
    print("=======================================")


Looping the algorithm for every courier (Adding the distances to the dataframe)

In [ ]:
distance_index_list = [0]*len(df)
df['Distance'] = distance_index_list
for i in set(df.CourierId):
    print("Current ID :",i)
    new_df = df.loc[df.CourierId == i]
    new_df = new_df.set_index([[i for i in range(len(new_df))]])
    dist_matrix = [[0]*len(new_df) for _ in range(len(new_df))]

    for j in new_df.index:
        for k in new_df.index:
            loc1=(new_df.Latitude[j], new_df.Longitude[j])
            loc2=(new_df.Latitude[k], new_df.Longitude[k])
            dist_matrix[j][k] = hs.haversine(loc1,loc2)
    
    dist_matrix = np.array(dist_matrix)

    arr1,rl_dis = solve_tsp_local_search(dist_matrix)
    
    for idx in df.index:
        if df.CourierId[idx] == i:
            df.Distance[idx] = rl_dis

Displaying the new Dataframe (distances included)

In [ ]:
df